In [9]:
import sys
sys.path.append('../scripts/')
from robot import *
from scipy.stats import multivariate_normal

In [10]:
class Particle:
    def __init__(self, init_pose):
        self.pose = init_pose
        
    def motion_update(self, nu, omega, time, noise_rate_pdf):
        ns = noise_rate_pdf.rvs()
        pnu = nu + ns[0]*math.sqrt(abs(nu)/time) + ns[1]*math.sqrt(abs(omega)/time)
        pomega = omega + ns[2]*math.sqrt(abs(nu)/time) + ns[3]*math.sqrt(abs(omega)/time)
        self.pose = IdealRobot.state_transition(pnu, pomega, time, self.pose)

In [11]:
class Mcl:
    def __init__(self, init_pose, num, motion_noise_stds):
        self.particles = [Particle(init_pose) for i in range(num)]
        
        v = motion_noise_stds
        c = np.diag([v["nn"]**2, v["no"]**2, v["on"]**2, v["oo"]**2])
        self.motion_noise_rate_pdf = multivariate_normal(cov=c)
        
    def motion_update(self, nu, omega, time):
        for p in self.particles: p.motion_update(nu, omega, time, self.motion_noise_rate_pdf)
        
    def draw(self, ax, elems):
        xs = [p.pose[0] for p in self.particles]
        ys = [p.pose[1] for p in self.particles]
        vxs = [math.cos(p.pose[2]) for p in self.particles]
        vys = [math.sin(p.pose[2]) for p in self.particles]
        elems.append(ax.quiver(xs, ys, vxs, vys, color='blue', alpha=0.5))

In [12]:
class EstimationAgent(Agent):
    def __init__(self, time_interval, nu, omega, estimator):
        super().__init__(nu, omega)
        self.estimator = estimator
        self.time_interval = time_interval
        
        self.prev_nu = 0.0
        self.prev_omega = 0.0
        
    def decision(self, observation=None):
        self.estimator.motion_update(self.prev_nu, self.prev_omega, self.time_interval)
        self.prev_nu, self.prev_omega = self.nu, self.omega
        self.estimator.observation_update(observation)
        return self.nu, self.omega
        
    def draw(self, ax, elems):
        self.estimator.draw(ax, elems)

In [13]:
initial_pose = np.array([0, 0, 0]).T 
estimator = Mcl(initial_pose, 100, motion_noise_stds={"nn":0.01, "no":0.02, "on":0.03, "oo":0.04})
a = EstimationAgent(0.1, 0.2, 10.0/180*math.pi, estimator)
estimator.motion_update(0.2, 10.0/180*math.pi, 0.1)
for p in estimator.particles:
    print(p.pose)

[0.02242636 0.00016295 0.01453158]
[0.02343106 0.00045291 0.03865437]
[0.02178346 0.00018513 0.0169968 ]
[0.01654745 0.00014858 0.0179579 ]
[0.02347408 0.00030706 0.02615996]
[0.02162253 0.00014855 0.01374015]
[1.84829909e-02 8.81154765e-05 9.53469160e-03]
[0.02107525 0.000336   0.03188278]
[1.92463268e-02 8.75421124e-05 9.09695751e-03]
[0.01936337 0.00015655 0.01616927]
[1.90536503e-02 7.42826246e-05 7.79716718e-03]
[0.01989932 0.00011324 0.01138164]
[0.02270012 0.00026344 0.02320965]
[0.02188921 0.0002333  0.02131592]
[0.01792245 0.00013622 0.01520103]
[2.01231697e-02 6.28234222e-05 6.24386903e-03]
[0.01723168 0.00025878 0.03003351]
[1.94643648e-02 6.88876556e-05 7.07830630e-03]
[0.02012173 0.00024886 0.02473463]
[0.01694346 0.00023698 0.02797067]
[1.40325752e-02 7.20271498e-05 1.02656164e-02]
[0.02238491 0.00018138 0.01620503]
[0.02082743 0.00019653 0.01887176]
[0.02580581 0.00032087 0.02486646]
[0.02380599 0.0001455  0.01222368]
[0.01841838 0.00018827 0.02044257]
[1.54825150e-02 8.

In [14]:
def trial(motion_noise_stds): 
    time_interval = 0.1 
    world = World(30, time_interval) 

    initial_pose = np.array([0, 0, 0]).T
    estimator = Mcl(initial_pose, 100, motion_noise_stds)
    circling = EstimationAgent(time_interval, 0.2, 10.0/180*math.pi, estimator)
    r = Robot(initial_pose, sensor=None, agent=circling, color="red" )
    world.append(r)

    world.draw()
    
trial({"nn":0.01, "no":0.02, "on":0.03, "oo":0.04})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …